# 4️⃣ Benchmark: 표준 벤치마크 평가

이 노트북은 다음을 수행합니다:
1. PTH 모델 벤치마크 평가 (Set5, Set14, B100, Urban100, Manga109)
2. PTL 모델 벤치마크 평가
3. 결과 저장 및 비교 시각화

**평가 대상:**
- 원본 CATANet-L x2 (PTH + PTL)
- Output KD (PTH + PTL)
- Feature KD (PTH + PTL)
- FaKD (PTH + PTL)

**논문 Table 2 (CATANet-L ×2) 기준값:**
- Set5: 38.28 dB
- Set14: 33.99 dB
- B100: 32.37 dB
- Urban100: 33.09 dB
- Manga109: 39.37 dB

---
## ⚙️ 0. 경로 설정 (이 셀만 수정하세요!)

In [ ]:
# ============================================================
# 🔧 사용자 설정 - 1_setup.ipynb와 동일하게!
# ============================================================
BASE_DIR = "my_ssl"
VENV_NAME = "ssl_env"
PYTHON_VERSION = "3.12"
REPO_NAME = "uos-ssl-project"

# ============================================================
# 🔒 아래는 자동 설정 (수정 불필요)
# ============================================================
import os
import sys
import subprocess
os.environ['MPLBACKEND'] = 'Agg'

home_dir = os.path.expanduser('~')
base_path = os.path.join(home_dir, BASE_DIR)
venv_path = os.path.join(base_path, VENV_NAME)
site_packages = os.path.join(venv_path, 'lib', f'python{PYTHON_VERSION}', 'site-packages')
python_exec = os.path.join(venv_path, 'bin', 'python')
project_dir = os.path.join(base_path, REPO_NAME)
ai_dir = os.path.join(project_dir, 'ai')

if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

%cd {ai_dir}

os.makedirs('results/benchmark', exist_ok=True)

# 경로 검증
print("="*60)
print("🔍 경로 검증")
print("="*60)
paths_ok = True
for name, path in [("AI Dir", ai_dir), ("Python", python_exec), ("Venv", venv_path)]:
    if os.path.exists(path):
        print(f"✅ {name}: {path}")
    else:
        print(f"❌ {name}: {path} (없음!)")
        paths_ok = False

if not paths_ok:
    raise RuntimeError("경로 설정 오류! 위의 사용자 설정을 확인하세요.")

print("="*60)

In [ ]:
# ============================================================
# 유틸리티 함수
# ============================================================
def run_benchmark(cmd, task_name):
    """벤치마크 명령 실행 후 에러 체크"""
    result = subprocess.run(cmd, cwd=ai_dir)
    
    if result.returncode != 0:
        print(f"❌ {task_name} 실패! (exit code: {result.returncode})")
        return False
    
    print(f"✅ {task_name} 완료!")
    return True

## 4.1 평가 대상 모델 확인

In [ ]:
print("="*70)
print("📁 평가 대상 모델 확인")
print("="*70)

pth_models = [
    ('Original', 'weights/CATANet-L_x2.pth'),
    ('Output_KD', 'weights/finetuned_output.pth'),
    ('Feature_KD', 'weights/finetuned_feature.pth'),
    ('FaKD', 'weights/finetuned_fakd.pth'),
]

print("\n🔹 PTH 모델:")
available_pth = []
for name, path in pth_models:
    full_path = os.path.join(ai_dir, path)
    if os.path.exists(full_path):
        size = os.path.getsize(full_path) / (1024 * 1024)
        print(f"   ✅ {name:12} | {path} ({size:.1f} MB)")
        available_pth.append((name, path))
    else:
        print(f"   ❌ {name:12} | {path} (없음)")

print("\n🔹 PTL 모델:")
weights_dir = os.path.join(ai_dir, 'weights')
ptl_count = 0
if os.path.exists(weights_dir):
    ptl_files = sorted([f for f in os.listdir(weights_dir) if f.endswith('.ptl')])
    for f in ptl_files:
        path = os.path.join(weights_dir, f)
        size = os.path.getsize(path) / (1024 * 1024)
        print(f"   ✅ {f} ({size:.1f} MB)")
        ptl_count += 1
    if ptl_count == 0:
        print("   ❌ PTL 파일 없음")

print(f"\n📊 PTH: {len(available_pth)}개, PTL: {ptl_count}개")

## 4.2 벤치마크 데이터셋 확인

In [ ]:
benchmark_dir = os.path.join(ai_dir, 'datasets/benchmark')

print("📁 벤치마크 데이터셋 확인:")
print("="*60)

expected_datasets = ['Set5', 'Set14', 'B100', 'Urban100', 'Manga109']
all_exist = True

if os.path.exists(benchmark_dir):
    for ds in expected_datasets:
        ds_path = os.path.join(benchmark_dir, ds)
        if os.path.exists(ds_path):
            print(f"   ✅ {ds}")
        else:
            print(f"   ❌ {ds} (없음)")
            all_exist = False
else:
    print("❌ 벤치마크 데이터셋 폴더가 없습니다!")
    print("   1_setup.ipynb를 먼저 실행하세요.")
    all_exist = False

if all_exist:
    print("\n✅ 모든 벤치마크 데이터셋 준비됨")

---
## 4.3 PTH 모델 벤치마크 평가

In [ ]:
print("="*70)
print("🎯 PTH 모델 벤치마크 평가")
print("="*70)
print("\n📊 논문 기준값 (CATANet-L ×2):")
print("   Set5: 38.28 | Set14: 33.99 | B100: 32.37 | Urban100: 33.09 | Manga109: 39.37")
print("="*70)

pth_models = [
    ('Original', 'weights/CATANet-L_x2.pth'),
    ('Output_KD', 'weights/finetuned_output.pth'),
    ('Feature_KD', 'weights/finetuned_feature.pth'),
    ('FaKD', 'weights/finetuned_fakd.pth'),
]

output_csv = 'results/benchmark/pth_benchmark_results.csv'
success_count = 0
fail_count = 0

for name, path in pth_models:
    full_path = os.path.join(ai_dir, path)
    if os.path.exists(full_path):
        print(f"\n{'#'*60}")
        print(f"# {name}: {path}")
        print(f"{'#'*60}")
        
        cmd = [
            python_exec, 'evaluate_benchmark.py',
            '--weights', path,
            '--benchmark_dir', 'datasets/benchmark',
            '--scale', '2',
            '--model_name', f'{name}_PTH',
            '--output', output_csv
        ]
        
        if run_benchmark(cmd, f"{name} 평가"):
            success_count += 1
        else:
            fail_count += 1
    else:
        print(f"\n⚠️ {name} 없음: {path}")

print("\n" + "="*70)
if fail_count == 0 and success_count > 0:
    print(f"✅ PTH 벤치마크 완료! ({success_count}개 모델)")
    print(f"   결과 저장: {output_csv}")
elif fail_count > 0:
    print(f"⚠️ PTH 벤치마크 일부 실패! ({success_count} 성공, {fail_count} 실패)")
else:
    print("⚠️ 평가할 PTH 모델이 없습니다.")

---
## 4.4 PTL 모델 벤치마크 평가

In [ ]:
print("="*70)
print("🎯 PTL 모델 벤치마크 평가")
print("="*70)

# PTL 모델 목록 수집
ptl_models = []
weights_dir = os.path.join(ai_dir, 'weights')

if os.path.exists(weights_dir):
    for f in os.listdir(weights_dir):
        if f.endswith('.ptl'):
            parts = f.replace('.ptl', '').split('_')
            try:
                input_size = int(parts[-1])
                model_name = '_'.join(parts[:-1])
                ptl_models.append((model_name, f'weights/{f}', input_size))
            except:
                pass

output_csv = 'results/benchmark/ptl_benchmark_results.csv'

if len(ptl_models) == 0:
    print("⚠️ PTL 파일이 없습니다!")
    print("   1_setup.ipynb와 3_ptl.ipynb를 먼저 실행하세요.")
else:
    ptl_models.sort(key=lambda x: (x[0], x[2]))
    
    success_count = 0
    fail_count = 0
    
    for model_name, ptl_path, input_size in ptl_models:
        print(f"\n{'#'*60}")
        print(f"# {model_name} (입력: {input_size}x{input_size})")
        print(f"{'#'*60}")
        
        cmd = [
            python_exec, 'evaluate_ptl_benchmark.py',
            '--ptl_path', ptl_path,
            '--input_size', str(input_size),
            '--benchmark_dir', 'datasets/benchmark',
            '--scale', '2',
            '--model_name', model_name,
            '--output', output_csv
        ]
        
        if run_benchmark(cmd, f"{model_name}_{input_size} 평가"):
            success_count += 1
        else:
            fail_count += 1
    
    print("\n" + "="*70)
    if fail_count == 0:
        print(f"✅ PTL 벤치마크 완료! ({success_count}개 모델)")
        print(f"   결과 저장: {output_csv}")
    else:
        print(f"⚠️ PTL 벤치마크 일부 실패! ({success_count} 성공, {fail_count} 실패)")

---
## 4.5 결과 비교 테이블

In [ ]:
import pandas as pd
from IPython.display import display

print("="*80)
print("📊 벤치마크 결과 비교")
print("="*80)

pth_results = os.path.join(ai_dir, 'results/benchmark/pth_benchmark_results.csv')
if os.path.exists(pth_results):
    print("\n🔹 PTH 모델 결과:")
    df_pth = pd.read_csv(pth_results)
    display(df_pth)
else:
    print("\n⚠️ PTH 벤치마크 결과 없음")

ptl_results = os.path.join(ai_dir, 'results/benchmark/ptl_benchmark_results.csv')
if os.path.exists(ptl_results):
    print("\n🔹 PTL 모델 결과:")
    df_ptl = pd.read_csv(ptl_results)
    display(df_ptl)
else:
    print("\n⚠️ PTL 벤치마크 결과 없음")

print("\n🔹 논문 기준값 (CATANet-L ×2):")
print("   Set5: 38.28 | Set14: 33.99 | B100: 32.37 | Urban100: 33.09 | Manga109: 39.37")

---
## 4.6 결과 시각화

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print("📊 벤치마크 결과 시각화")
print("="*60)

pth_results = os.path.join(ai_dir, 'results/benchmark/pth_benchmark_results.csv')
if os.path.exists(pth_results):
    df = pd.read_csv(pth_results)
    
    paper = {'Set5': 38.28, 'Set14': 33.99, 'B100': 32.37, 'Urban100': 33.09, 'Manga109': 39.37}
    datasets = ['Set5', 'Set14', 'B100', 'Urban100', 'Manga109']
    
    fig, ax = plt.subplots(figsize=(14, 6))
    x = np.arange(len(datasets))
    width = 0.15
    
    paper_vals = [paper[ds] for ds in datasets]
    ax.bar(x - width*2, paper_vals, width, label='Paper', color='gray', alpha=0.7)
    
    colors = ['#3B82F6', '#10B981', '#F59E0B', '#EF4444']
    for idx, (_, row) in enumerate(df.iterrows()):
        if idx >= len(colors):
            break
        vals = []
        for ds in datasets:
            col = f'{ds}_PSNR'
            try:
                vals.append(float(row[col]) if col in row and pd.notna(row[col]) else 0)
            except:
                vals.append(0)
        ax.bar(x + width*(idx-1), vals, width, label=row['Model'], color=colors[idx])
    
    ax.set_ylabel('PSNR (dB)')
    ax.set_xlabel('Dataset')
    ax.set_title('PTH Model Benchmark')
    ax.set_xticks(x)
    ax.set_xticklabels(datasets)
    ax.legend(loc='lower right')
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(30, 42)
    
    plt.tight_layout()
    save_path = os.path.join(ai_dir, 'results/benchmark/pth_comparison.png')
    plt.savefig(save_path, dpi=150)
    plt.show()
    print(f"\n✅ 저장: {save_path}")
else:
    print("⚠️ PTH 결과 없음")

ptl_results = os.path.join(ai_dir, 'results/benchmark/ptl_benchmark_results.csv')
if os.path.exists(ptl_results):
    df_ptl = pd.read_csv(ptl_results)
    if len(df_ptl) > 0 and 'Avg_Time_ms' in df_ptl.columns:
        fig, ax = plt.subplots(figsize=(12, 6))
        models = df_ptl['Model'].unique()
        colors = plt.cm.Set2(np.linspace(0, 1, len(models)))
        
        for idx, model in enumerate(models):
            data = df_ptl[df_ptl['Model'] == model].sort_values('Input_Size')
            ax.plot(data['Input_Size'], data['Avg_Time_ms'], 
                    marker='o', label=model, color=colors[idx], linewidth=2)
        
        ax.set_xlabel('Input Size')
        ax.set_ylabel('Inference Time (ms)')
        ax.set_title('PTL Inference Time')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        save_path = os.path.join(ai_dir, 'results/benchmark/ptl_inference_time.png')
        plt.savefig(save_path, dpi=150)
        plt.show()
        print(f"\n✅ 저장: {save_path}")

---
## 4.7 최종 요약

In [ ]:
import pandas as pd

print("="*80)
print("📊 최종 벤치마크 요약")
print("="*80)

results_dir = os.path.join(ai_dir, 'results/benchmark')
print("\n📁 생성된 결과 파일:")
if os.path.exists(results_dir):
    files = os.listdir(results_dir)
    if files:
        for f in sorted(files):
            path = os.path.join(results_dir, f)
            size = os.path.getsize(path) / 1024
            print(f"   {f} ({size:.1f} KB)")
    else:
        print("   없음")
else:
    print("   없음")

pth_results = os.path.join(ai_dir, 'results/benchmark/pth_benchmark_results.csv')
if os.path.exists(pth_results):
    df = pd.read_csv(pth_results)
    print("\n🔹 PTH 모델 Set5 PSNR:")
    for _, row in df.iterrows():
        if 'Set5_PSNR' in row and pd.notna(row['Set5_PSNR']):
            try:
                print(f"   {row['Model']:20} | {float(row['Set5_PSNR']):.4f} dB")
            except:
                pass

ptl_results = os.path.join(ai_dir, 'results/benchmark/ptl_benchmark_results.csv')
if os.path.exists(ptl_results):
    df = pd.read_csv(ptl_results)
    if 'Input_Size' in df.columns and 'Avg_Time_ms' in df.columns:
        print("\n🔹 PTL 추론 시간 (640x640):")
        df_640 = df[df['Input_Size'] == 640]
        for _, row in df_640.iterrows():
            print(f"   {row['Model']:20} | {row['Avg_Time_ms']:.2f} ms")

print("\n" + "="*80)
print("✅ 벤치마크 완료!")
print("="*80)